# 1. Import Required Libraries (Run for all!)
Import the necessary libraries, including pandas, nibabel, nilearn, and numpy.

In [3]:
# Import Required Libraries

import pandas as pd  # For data manipulation
import nibabel as nib  # For loading fMRI data
from nilearn import plotting, image, glm  # For fMRI data analysis and visualization
import numpy as np  # For numerical operations
import os
import glob

In [11]:
# Define the base directory
base_dir = '/Users/hannahmueller/nyx_mount/Downloads/ibc_all/'

In [48]:
# Define the subject ID and task type
Subject = '01'  # CHANGE
Task = 'ArchiSocial' # CHANGE

# 2. BIDS Structure

my data is in bids structure where each fmri file is named like this sub-XX_ses-YY_task-ZZZ_dir-AA_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz. Help me browse through all my folders in "/Users/hannahmueller/nyx_mount/Downloads/ibc_all/" to find all files i have when inputting the subject ID XX and task type ZZZ

In [ ]:
# Function to find out when and how often a specific task was done by a subject.

def find_fmri_files(base_dir, Subject, Task):
    # Define the search pattern
    search_pattern = f"sub-{Subject}_ses-*_task-{Task}_dir-*_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
    
    # Use glob to find all matching files
    fmri_files = glob.glob(os.path.join(base_dir, '**', search_pattern), recursive=True)
    
    return fmri_files

# Find the fMRI files
fmri_files = find_fmri_files(base_dir, Subject, Task)

# Display the found files
fmri_files

['/Users/hannahmueller/nyx_mount/Downloads/ibc_all/sub-01/ses-07/func/sub-01_ses-07_task-ArchiSocial_dir-ap_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
 '/Users/hannahmueller/nyx_mount/Downloads/ibc_all/sub-01/ses-07/func/sub-01_ses-07_task-ArchiSocial_dir-pa_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
 '/Users/hannahmueller/nyx_mount/Downloads/ibc_all/sub-01/ses-00/func/sub-01_ses-00_task-ArchiSocial_dir-pa_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
 '/Users/hannahmueller/nyx_mount/Downloads/ibc_all/sub-01/ses-00/func/sub-01_ses-00_task-ArchiSocial_dir-ap_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz']

In [83]:
# Function to create task overview dataframe

def create_task_overview(base_dir):
    # Define the search pattern for the fMRI files
    search_pattern = f"sub-{Subject}_ses-*_task-*_dir-ap_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
    
    # Use glob to find all matching files
    fmri_files = glob.glob(os.path.join(base_dir, '**', search_pattern), recursive=True)
    
    # Create a dataframe to store the task overview
    task_overview = pd.DataFrame(fmri_files, columns=['File'])
    
    # Extract task information from the file paths
    task_overview['Task'] = task_overview['File'].apply(lambda x: os.path.basename(x).split('_')[2].split('-')[1])
    
    # Extract session information from the file paths
    task_overview['Session'] = task_overview['File'].apply(lambda x: os.path.basename(x).split('_')[1])
    
    return task_overview

# Create the task overview dataframe
task_overview = create_task_overview(base_dir)

# Display the task overview dataframe
task_overview



,File,Task,Session
0,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,HcpLanguage,ses-03
1,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,HcpGambling,ses-03
2,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,HcpMotor,ses-03
3,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,HcpEmotion,ses-03
4,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,Audi,ses-23
5,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,Lec2,ses-23
6,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,Lec1,ses-23
7,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,Visu,ses-23
8,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,ArchiStandard,ses-07
9,/Users/hannahmueller/nyx_mount/Downloads/ibc_a...,ArchiSocial,ses-07


In [52]:
# Extract task information from the file paths
task_overview['Task'] = task_overview['File'].apply(lambda x: os.path.basename(x).split('_')[2].split('-')[1])

# Extract session information from the file paths
task_overview['Session'] = task_overview['File'].apply(lambda x: os.path.basename(x).split('_')[1])

# Group by task and list the unique session folders for each task
task_sessions = task_overview.groupby('Task')['Session'].unique().reset_index()

# Display the task sessions overview
task_sessions

,Task,Session
0,ArchiEmotional,"[ses-01, ses-04, ses-07]"
1,ArchiSocial,"[ses-00, ses-01, ses-04, ses-07]"
2,ArchiSpatial,"[ses-00, ses-01, ses-04, ses-07]"
3,ArchiStandard,"[ses-00, ses-01, ses-04, ses-07]"
4,Attention,"[ses-26, ses-24]"
...,...,...
68,Visu,"[ses-22, ses-19, ses-23]"
69,VisualSearch,"[ses-39, ses-38]"
70,WardAndAllport,"[ses-28, ses-26]"
71,WedgeAnti,"[ses-08, ses-11, ses-09]"


Do the same as for task_sessions but in a common dataset for all subjects to show for all tasks (columns) where it's shown in which folders/sessions this task appered. Sort by subject (rows) and task (columns)

# 3. Tasks Conditions/Contrast
File naming: sub-{Subject}_ses-*_task-{Task}_dir-ap_events.tsv

In [69]:
# Choose Subject and Task
Subject = '01'
Task = 'ArchiStandard'

NameError: name 'ArchiStandard' is not defined

In [77]:
# Open the overview file for all tasks and sessions
task_overview = pd.read_csv('common_tasks.csv')

# Remove .0 from any float columns
task_overview = task_overview.map(lambda x: int(x) if isinstance(x, float) and x.is_integer() else x)

task_overview

,Subject,ArchiEmotional,ArchiSocial,ArchiSpatial,ArchiStandard,Attention,Audi,Audio,Bang,BiologicalMotion1,...,Stroop,TheoryOfMind,TwoByTwo,VSTM,VSTMC,Visu,VisualSearch,WardAndAllport,WedgeAnti,WedgeClock
0,1,7,"07, 00","07, 00","07, 00",NaN,23.0,NaN,24.0,NaN,...,NaN,18.0,NaN,20.0,NaN,23.0,NaN,NaN,11.0,11.0
1,2,1,"00, 01","00, 01","00, 01",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,4,04,"00, 04","00, 04",24.0,19.0,"23, 22",21.0,29.0,...,25.0,16.0,24.0,17.0,36.0,19.0,38.0,26.0,9.0,9.0
3,5,4,04,"00, 04","00, 04",24.0,19.0,"23, 21",20.0,29.0,...,25.0,16.0,24.0,17.0,36.0,19.0,38.0,26.0,9.0,9.0
4,6,4,"00, 04","00, 04","00, 04",24.0,19.0,"23, 22",21.0,29.0,...,25.0,16.0,24.0,17.0,36.0,19.0,38.0,26.0,8.0,8.0
5,7,4,"00, 04","00, 04","00, 04",24.0,19.0,"21, 22",23.0,29.0,...,25.0,16.0,24.0,17.0,36.0,19.0,NaN,26.0,8.0,8.0
6,8,1,"00, 01","00, 01","00, 01",26.0,22.0,"24, 25",23.0,31.0,...,27.0,19.0,26.0,20.0,37.0,22.0,39.0,28.0,8.0,8.0


In [ ]:
# Extract file path for the chosen subject and task


UndefinedVariableError: name 'Task' is not defined

In [28]:
# Sanity Check. Are all .tsv files the same across sessions for a given subject and task?

def compare_tsv_files(base_dir, Subject, Task):
    # Define the search pattern for the events files
    search_pattern = f"sub-{Subject}_ses-*_task-{Task}_dir-ap_events.tsv"
    
    # Use glob to find all matching files
    tsv_files = glob.glob(os.path.join(base_dir, '**', search_pattern), recursive=True)
    
    # Load the content of each tsv file into a list of dataframes
    tsv_data = [pd.read_csv(tsv_file, sep='\t') for tsv_file in tsv_files]
    
    # Compare the dataframes to check if they are the same
    comparison_results = all(tsv_data[0].equals(df) for df in tsv_data[1:])
    
    return comparison_results, tsv_files

# Compare the .tsv files across session folders
comparison_results, tsv_files = compare_tsv_files(base_dir, Subject, Task)

# Display the comparison results
print(f"Are all .tsv files the same across sessions for subject {Subject} and task {Task}? {comparison_results}")
print("Files compared:")
for tsv_file in tsv_files:
    print(tsv_file)

Are all .tsv files the same across sessions for subject 01 and task ArchiStandard? True
Files compared:
/Users/hannahmueller/nyx_mount/Downloads/ibc_all/sub-01/ses-07/func/sub-01_ses-07_task-ArchiStandard_dir-ap_events.tsv
/Users/hannahmueller/nyx_mount/Downloads/ibc_all/sub-01/ses-00/func/sub-01_ses-00_task-ArchiStandard_dir-ap_events.tsv


## 3.1 Load Event Data
Load the event data from the specified .tsv file using pandas.

In [34]:
# Load Event Data
events_file_path = glob.glob(os.path.join(base_dir, f'sub-{Subject}', '**', f'sub-{Subject}_ses-*_task-{Task}_dir-ap_events.tsv'), recursive=True)[0]

# Load the event data using pandas
events_data = pd.read_csv(events_file_path, sep='\t')

# Display the first few rows of the event data to verify it has been loaded correctly
events_data.head()

,onset,duration,trial_type
0,0.0,1,video_computation
1,2.4,1,video_computation
2,8.7,1,horizontal_checkerboard
3,11.4,1,audio_right_hand
4,15.0,1,audio_sentence


## Find Trial Types
Preprocess the event data to extract different conditions of the task.

In [45]:
# Preprocess Event Data

# Extract different conditions of the task from the event data
conditions = events_data['trial_type'].unique()

# Create a dictionary to store the onset times for each condition
condition_onsets = {condition: events_data[events_data['trial_type'] == condition]['onset'].values for condition in conditions}

# Display the extracted conditions and their corresponding onset times
condition_onsets

{'video_computation': array([  0. ,   2.4, 125.4, 153. , 164.4, 198. , 201. , 221.4, 234. ,
        260.4]),
 'horizontal_checkerboard': array([  8.7,  59.7, 149.4, 176.7, 188.4, 218.7, 224.7, 248.4, 266.7,
        269.7]),
 'audio_right_hand': array([ 11.4,  87. , 143.4, 162. , 230.7]),
 'audio_sentence': array([ 15. ,  20.7,  29.7,  89.7, 119.4, 146.7, 236.7, 284.4, 293.4,
        296.7]),
 'video_right_hand': array([ 18. ,  69. , 227.7, 275.4, 291. ]),
 'audio_left_hand': array([ 23.7,  62.4, 170.4, 254.7, 257.4]),
 'video_left_hand': array([ 26.7,  71.4, 116.7, 212.7, 215.7]),
 'vertical_checkerboard': array([ 33. ,  96. , 122.7, 140.4, 156. , 203.7, 207. , 210. , 264. ,
        278.4]),
 'audio_computation': array([ 35.4,  44.7,  48. ,  83.4, 108. , 135. , 137.7, 173.7, 191.7,
        251.7]),
 'video_sentence': array([ 39. ,  41.7,  56.4,  75. , 131.4, 159. , 167.7, 195. , 246. ,
        288. ])}

## Load fMRI Data
Load the fMRI data from the .nii file using nibabel.

In [ ]:
# Load fMRI Data

# Define the path to the fMRI data file
fmri_file_path = '/Users/hannahmueller/nyx_mount/Downloads/ibc_all/sub-01/ses-00/func/sub-01_ses-00_task-ArchiSocial_dir-ap_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

# Load the fMRI data using nibabel
fmri_img = nib.load(fmri_file_path)

# Display basic information about the fMRI data
print(fmri_img)

# Plot the fMRI data to visualize it
plotting.plot_img(fmri_img, title="fMRI Data")


<class 'nibabel.nifti1.Nifti1Image'>
data shape (105, 127, 105, 262)
affine:
[[  -1.5    0.     0.    78. ]
 [   0.     1.5    0.  -112. ]
 [   0.     0.     1.5  -70. ]
 [   0.     0.     0.     1. ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : np.bytes_(b'')
db_name         : np.bytes_(b'')
extents         : 0
session_error   : 0
regular         : np.bytes_(b'r')
dim_info        : 0
dim             : [  4 105 127 105 262   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.   1.5  1.5  1.5  0.   0.   0.   0. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : np.

# GLM

## Design Matrix Creation
Create a design matrix for the GLM using the preprocessed event data.

In [ ]:
# Design Matrix Creation

from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
import matplotlib.pyplot as plt

# Define the repetition time (TR) of the fMRI data
tr = 2.0  # Example TR, replace with the actual TR of your fMRI data

# Define the frame times based on the number of volumes and TR
n_scans = fmri_img.shape[-1]
frame_times = np.arange(n_scans) * tr

# Create the design matrix using the condition onsets
design_matrix = make_first_level_design_matrix(frame_times, condition_onsets)

# Display the design matrix
plot_design_matrix(design_matrix)
plt.show()

# Display the first few rows of the design matrix
design_matrix.head()

## Fit GLM Model
Fit the GLM model to the fMRI data using nilearn's GLM functions.

In [ ]:
from nilearn.glm.first_level import FirstLevelModel

# Fit GLM Model

# Initialize the GLM model
glm_model = FirstLevelModel(t_r=tr, slice_time_ref=0.5, hrf_model='spm')

# Fit the GLM model to the fMRI data using the design matrix
glm_model = glm_model.fit(fmri_img_normalized, design_matrices=design_matrix)

# Display the summary of the fitted model
glm_model

## Visualize Results
Visualize the results of the GLM analysis, including statistical maps and activation patterns.

In [ ]:
# Visualize Results

# Compute the contrast for each condition
from nilearn.glm.contrasts import compute_contrast

# Define the contrast for each condition (e.g., condition vs. baseline)
contrasts = {condition: np.eye(len(conditions))[i] for i, condition in enumerate(conditions)}

# Compute the contrast maps
contrast_maps = {condition: glm_model.compute_contrast(contrasts[condition], output_type='z_score') for condition in conditions}

# Plot the contrast maps
for condition, contrast_map in contrast_maps.items():
    plotting.plot_stat_map(contrast_map, title=f'Contrast Map: {condition}', threshold=3.0, display_mode='z', cut_coords=10)
    plotting.show()

# Display the activation patterns for each condition
for condition, contrast_map in contrast_maps.items():
    plotting.plot_glass_brain(contrast_map, title=f'Activation Pattern: {condition}', threshold=3.0, display_mode='lyrz')
    plotting.show()